# Building Dataset. Part 1
### By **Néstor Suat** in 2020

**Descripción:** Contruyendo el dataset Parte 1. Generar los archivos para BRAT Annotation Tools.

**Input:**
* TSV con los tweets seleccionados

**Output:**
* Files *.txt and *.ann
***

### Importando librerías

Como estamos en un archivo afuera se necesita agregar la dirección ../ (raíz del proyexto) para importar la librería de preprocesamiento.

In [1]:
import pandas as pd
import spacy  # For preprocessing
import re

import sys
sys.path.insert(0, '../../../')

from classes.wordsegmentation import WordSegmentation

In [2]:
nlp = spacy.load("es_core_news_lg") # disabling Named Entity Recognition for speed

### Importando datasets

In [3]:
file = 'ner_dataset_shuffle.tsv'
dir_ = "../../../data/v1/NER/"

In [4]:
dataset = pd.read_csv(dir_+file, delimiter = "\t", quoting = 3)
del dataset['Unnamed: 0']
dataset = dataset['text']
dataset[0]

'#ElNoctámbulo Aparatoso accidente en la Avenida Suba con calle 127. El conductor se salvó de milagro. #ArribaBogotá https://t.co/D6qLLsufhG'

In [5]:
dataset.shape

(1340,)

In [6]:
#Seleccionando 50
#df = dataset.sample(n=50)
df = dataset.sample(frac=1)

### Aplicando limpieza

In [7]:
seg = WordSegmentation(dir_+'spanish_count_1w_small_v2_twitter.txt')

In [8]:
def word_segmentation(pattern, text):
    search = re.search(pattern,text)
    while search:    
        s = search.start()
        e = search.end()                
        text = text[:s] + ' ' + ' '.join(seg.segment(text[s+1:e])) +' '+ text[e:]        
        search = re.search(pattern,text)        
    return text

def clean_fn(doc):    
    text = doc
    pattern = "(@[A-Za-z0-9äÄëËïÏöÖüÜáéíóúáéíóúÁÉÍÓÚÂÊÎÔÛâêîôûàèìòùÀÈÌÒÙñÑ_]+)"
    text = word_segmentation(pattern,text)
    pattern = "(#[A-Za-z0-9äÄëËïÏöÖüÜáéíóúáéíóúÁÉÍÓÚÂÊÎÔÛâêîôûàèìòùÀÈÌÒÙñÑ_]+)"
    text = word_segmentation(pattern,text)
    return text
    
def preText(text):        
    pre = re.sub("&[A-Za-z]+;", ' ', text) #Eliminar códigos ASCII
    pre = re.sub("(\w+:\/\/\S+)",' ',pre) #Eliminar links http y https
    pre = re.sub("([^A-Za-z0-9äÄëËïÏöÖüÜáéíóúáéíóúÁÉÍÓÚÂÊÎÔÛâêîôûàèìòùÀÈÌÒÙñÑ,;.:*\-\[\]¿?¡!\"\"()_'/])",' ',pre) #Eliminar caracteres especiales como emoticones, exceptuando los signos de puntuación y tildes.
    pre = re.sub(r'([;.:\-\[\]¿?¡!#\"()]){3,}',r'\1\1',pre) #Si repite un caracters especial más de 3 veces ej. """"
    pre = re.sub(r'([a-zA-Z])\1{2,}',r'\1\1',pre) #Si repite una letra más de dos veces las reduce a dos repeticiones goool => gool        
    pre = re.sub(r'(\s){2,}',r' ',pre) #Eliminar espacios seguidos              
    return pre.strip()

In [9]:
brief_cleaning = (clean_fn(str(row)) for row in df)
txt = [preText(doc.text) for doc in nlp.pipe(brief_cleaning, batch_size=50, n_threads=4)]

In [10]:
txt

['Grave accidente en la autopista norte con calle 153, (N-S), una persona falleció y tres más heridas, fuerte congestión caracolradio',
 'Accidente de particular con volqueta en la Av ciudad de Cali en el semáforo de la calle 68 sentido norte sur. sectormovilidad waze traffic bog bogotatransito',
 'Accidente entre taxi y ambulancia en estación Santa Isabel de la Troncal NQS sur. La flota de TransMilenio transita por el carril mixto.',
 'bogotatransito sectormovilidad transitobta En el choque en la estacion Olaya que paralizó 4 carriles por una hora por que no asignaron unidades ni informaron por este medio??',
 'sectormovilidad bogotatransito Se solicita transito policia por accidente en la Autosur sentido norte - sur frente a Alfagres entre una volqueta y un taxi. No reportan heridos.',
 'trancon cll13 También hubo un choque múltiple frente a zona franca',
 'Hoy viernes, accidente en carrera 5 calle 69. Ahí había un reductor de velocidad pero ya no está. Por favor umv bogota necesitam

In [11]:
new = pd.DataFrame(df)
new['pre'] = txt
#new.to_csv('50_tweets_test.tsv',sep='\t')
new.to_csv('complete_tweets_ner_comparation.tsv',sep='\t')

### Generando archivos

In [12]:
#Generando txt y ann
i=1
for tweet in txt:
    name = dir_+'brat/'+'tweet-'+str(i)+'.txt'
    #print(name)  
    with open(name, "w") as file:
        file.write(tweet)
    name = dir_+'brat/'+'tweet-'+str(i)+'.ann'
    with open(name, "w") as file:
        file.write('')
    i+=1

In [13]:
txt[18]

'transmilenio ciclista accidentado en el carril exclusivo, estación Madelena. URGENTE!'